In [ ]:
import os
import pdfplumber
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

# Configuration
BASE_DIR = os.path.abspath(r"C:\Users\kosmo\pycode\MCP_Virginie")

# Initialiser le LLM
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ ATTENTION: Utilisez une variable d'environnement pour la clé API en production")
    api_key = "sk-proj-0xUZ6aBpi14QWLtzQC2nF0B2gQTojxukve0byW1qgx05yCT3BlbkFJ NH0hRFwAy9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=api_key
)

# ===== STATE DEFINITION =====

class PDFAnalysisState(TypedDict):
    pdf_filename: str
    start_page: Optional[int]
    end_page: Optional[int]
    extracted_text: Optional[str]
    extracted_file_path: Optional[str]
    analysis_result: Optional[str]
    error: Optional[str]

# ===== EXTRACTION FUNCTIONS =====

def post_process_text(text):
    """Fonction pour nettoyer et reformater le texte extrait."""
    # Corriger "A\nrticle" en "Article"
    text = re.sub(r'A\s*\n\s*rticle\s+(\d+)', r'Article \1', text)
    
    # Corriger "E\nn cas" en "En cas"
    text = re.sub(r'E\s*\n\s*n cas', r'En cas', text)
    
    # Corriger les sauts de lignes intempestifs dans le texte courant
    text = re.sub(r'(\w)\s*\n\s*(\w)', lambda m: f"{m.group(1)} {m.group(2)}" if not (m.group(2).lower() == 'les' or m.group(1) == '-') else f"{m.group(1)}\n{m.group(2)}", text)
    
    return text

def format_articles(text):
    """Améliore le formatage des articles pour respecter la mise en page désirée"""
    # Mettre "Article XX" sur une ligne séparée et ajouter un saut de ligne après
    text = re.sub(r'\b(Article\s+\d+)\s+', r'\n\1\n\n', text)
    
    # Assurer que "Direction de" commence toujours sur une nouvelle ligne
    text = re.sub(r'([^\n])(Direction de)', r'\1\n\n\2', text)
    
    # Assurer que chaque élément commençant par un tiret est sur une nouvelle ligne
    text = re.sub(r'([^\n])\s*(-\s+)', r'\1\n\2', text)
    
    # Assurer que "dans la limite de ses attributions et fonctions :" est sur sa propre ligne
    text = re.sub(r'(dans la limite de ses attributions et fonctions\s*:)', r'\n\1\n', text)
    
    # S'assurer que le texte après un tiret soit bien séparé de la Direction qui suit
    text = re.sub(r'(afférents\.)\s*(Direction)', r'\1\n\n\2', text)
    text = re.sub(r'(comptes\.)\s*(Direction)', r'\1\n\n\2', text)
    
    # S'assurer que "Bulletin officiel" est au début d'une ligne
    text = re.sub(r'([^\n])(Bulletin officiel)', r'\1\n\2', text)
    
    # Nettoyer les lignes vides multiples (pas plus de 2 consécutives)
    text = re.sub(r'\n{3,}', r'\n\n', text)
    
    return text

def extract_pdf_text(pdf_path: str, output_dir: str = "extracted_text", start_page: int = None, end_page: int = None):
    """Extract text from specified pages of a PDF file and save it to a single file."""
    if not os.path.exists(pdf_path):
        return None, f"Erreur: Fichier non trouvé: {pdf_path}"
    
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Déterminer la plage de pages
    if not end_page:
        end_page = float('inf')
        
    if not start_page:
        start_page = 1
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            
            # Ajuster la plage de pages si nécessaire
            start_idx = max(0, start_page - 1)
            end_idx = min(total_pages, end_page)
            
            # Préparer un seul fichier pour toutes les pages
            pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
            output_filename = f"{pdf_name}_pages_{start_page}-{end_idx}.txt"
            output_path = os.path.join(output_dir, output_filename)
            
            # Extraire et écrire le texte de toutes les pages dans un seul fichier
            extracted_text = ""
            with open(output_path, 'w', encoding='utf-8') as output_file:
                for i in range(start_idx, end_idx):
                    page = pdf.pages[i]
                    page_text = page.extract_text()
                    
                    if page_text:
                        # Post-traitement pour corriger les problèmes de formatage
                        page_text = post_process_text(page_text)
                        
                        # Amélioration du formatage des articles
                        page_text = format_articles(page_text)
                        
                        # Écrire un séparateur de page clair
                        page_content = f"\n\n{'='*20} PAGE {i+1} {'='*20}\n\n{page_text}\n"
                        output_file.write(page_content)
                        extracted_text += page_content
            
            return output_path, extracted_text
            
    except Exception as e:
        return None, f"Erreur lors de l'extraction du texte: {str(e)}"

# ===== LANGGRAPH NODES =====

def extraction_node(state: PDFAnalysisState) -> PDFAnalysisState:
    """Premier nœud : Extraction du PDF"""
    print("🔍 Exécution du nœud d'extraction...")
    
    pdf_filename = state["pdf_filename"]
    start_page = state.get("start_page")
    end_page = state.get("end_page")
    
    # Construire le chemin complet du PDF
    pdf_paths_to_try = [
        pdf_filename,
        os.path.join(BASE_DIR, pdf_filename),
        os.path.join(os.path.dirname(os.path.abspath(__file__)), pdf_filename)
    ]
    
    pdf_path = None
    for path in pdf_paths_to_try:
        if os.path.exists(path):
            pdf_path = path
            break
    
    if not pdf_path:
        return {
            **state,
            "error": f"Fichier PDF non trouvé: {pdf_filename}\nChemins testés: {pdf_paths_to_try}"
        }
    
    # Extraction
    output_dir = os.path.join(BASE_DIR, "textes_extraits")
    extracted_file_path, extracted_text = extract_pdf_text(pdf_path, output_dir, start_page, end_page)
    
    if not extracted_file_path:
        return {
            **state,
            "error": extracted_text  # extracted_text contient l'erreur dans ce cas
        }
    
    print(f"✅ Extraction terminée. Fichier sauvé: {extracted_file_path}")
    
    return {
        **state,
        "extracted_file_path": extracted_file_path,
        "extracted_text": extracted_text
    }

def analysis_node(state: PDFAnalysisState) -> PDFAnalysisState:
    """Deuxième nœud : Analyse du texte extrait"""
    print("🔬 Exécution du nœud d'analyse...")
    
    if state.get("error"):
        return state
    
    extracted_text = state.get("extracted_text")
    if not extracted_text:
        return {
            **state,
            "error": "Aucun texte extrait à analyser"
        }
    
    try:
        # Prompt d'analyse (version simplifiée du prompt original)
        prompt_text = f"""Vous êtes un expert en analyse de texte juridique et administratif.
Votre spécialité est d'identifier les noms de personnes et leurs rôles
dans des documents officiels, particulièrement les délégations de pouvoir.

Analyse le texte et extrais les informations demandées, en suivant les étapes détaillées ci-dessous.

Chain of Thought pour l'analyse

Instructions générales :
Cette méthode d'analyse doit être appliquée sur l'ensemble du document administratif en procédant article par article. Pour chaque article du document :
1. Isoler le texte de l'article concerné
2. Appliquer la méthode d'analyse décrite dans les exemples ci-dessous
3. Rechercher systématiquement les termes spécifiques et les noms associés
4. Si les termes recherchés n'existent pas dans l'article, indiquer "Non mentionné" pour la catégorie correspondante
5. Compiler les résultats en respectant le format suivant pour chaque article :

Article [Numéro]
[En cas d'absence ou d'empêchement de] : [Nom de la personne ou "Non mentionné"]
[Délégation est donnée à] : [Nom de la personne ou "Non mentionné"]

Exemple 1:

1. Lecture du document :
   Je commence par lire attentivement l'extrait suivant pour en comprendre le contexte et la structure.

   "Délégation est donnée à M. Thomas DUPONT, responsable de l'Unité budget et contrôle interne au sein de la

   Direction des achats et des finances, à l'effet de signer, au nom de la directrice générale de Santé publique France,
   dans la limite de ses attributions et fonctions :

   - l'ensemble des bons de commande d'un montant hors taxe inférieur à 35 000 € ;
   - en cas d'absence ou d'empêchement de la directrice des achats et des finances,
   Mme Sophie MARTIN (épouse DURAND), l'ensemble des bons de commande ;
   - les certifications de service fait sans limitation de montant."

2. Recherche du terme "délégation est donnée" :
   Je cherche si cette expression apparaît dans le texte et j'examine ce qui suit.

   Trouvé : "Délégation est donnée à M. Thomas DUPONT"

   J'identifie donc le nom qui suit cette expression : M. Thomas DUPONT

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente :

   j'identifie la fonction devant le nom de M. Thomas DUPONT : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

   la fonction est :  responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

3. Recherche du terme "en cas d'absence ou d'empêchement" :
   Je cherche si cette expression apparaît dans le texte et j'examine le contexte.

   Trouvé : "en cas d'absence ou d'empêchement de la directrice des achats et des finances, Mme Sophie MARTIN (épouse DURAND)"

   Dans ce contexte, je comprends que Mme Sophie MARTIN (épouse DURAND) est mentionnée comme la directrice des achats et des finances.

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Sophie MARTIN (épouse DURAND) : directrice des achats et des finances

   la fonction est : directrice des achats et des finances

4. Format de sortie :
   Je structure les informations collectées selon le format demandé.

   [En cas d'absence ou d'empêchement de] : Mme Sophie MARTIN (épouse DURAND) , fonction : directrice des achats et des finances
   [Délégation est donnée à] : M. Thomas DUPONT , fonction : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

Exemple 2:

1. Lecture du document :
   Je commence par lire attentivement l'extrait suivant pour en comprendre le contexte et la structure.

   "En cas d'absence ou d'empêchement de Mme Sophia DUBOIS, directrice de l'aide et diffusion aux publics, délégation est donnée à Mme Camille LAURENT, adjointe, dans la limite de ses attributions et fonctions :

   - les engagements financiers relatifs à l'activité de la

   Direction de l'aide et diffusion aux publics d'un montant hors taxe inférieur à 25 000 € et les engagements contractuels afférents ;
   - les lettres de mission envoyées à des collaborateurs externes pour la relecture de rapports produits par Santé publique France avec la mention du montant d'indemnisation de la vacation ;
   - toute décision relative aux opérations d'inventaire dans le cadre de l'arrêté annuel des comptes."

2. Recherche du terme "En cas d'absence ou d'empêchement de" :
   Je cherche cette expression dans le texte et j'examine ce qui suit.

   Trouvé : "En cas d'absence ou d'empêchement de Mme Sophia DUBOIS"

   J'identifie donc le nom qui suit cette expression : Mme Sophia DUBOIS

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Sophia DUBOIS : directrice de l'aide et diffusion aux publics

   la fonction est : directrice de l'aide et diffusion aux publics

3. Recherche du terme "délégation est donnée à" :
   Je cherche cette expression dans le texte et j'examine ce qui suit.

   Trouvé : "délégation est donnée à Mme Camille LAURENT"

   J'identifie donc le nom qui suit cette expression : Mme Camille LAURENT

   Si cette expression n'était pas présente, j'indiquerais "Non mentionné".

   Si cette expression est présente j'identife également la fonction devant le nom de Mme Camille LAURENT : adjointe

   la fonction est : adjointe

4. Format de sortie :
   Je structure les informations collectées selon le format demandé.

   [En cas d'absence ou d'empêchement de] : Mme Sophia DUBOIS fonction : directrice de l'aide et diffusion aux publics
   [Délégation est donnée à] : Mme Camille LAURENT fonction : adjointe

Le résultat final pour l'ensemble du document ressemblera à ceci :

Article 22
[En cas d'absence ou d'empêchement de] : Mme Sophia DUBOIS fonction : directrice de l'aide et diffusion aux publics
[Délégation est donnée à] : Mme Camille LAURENT fonction : adjointe

Article 8
[En cas d'absence ou d'empêchement de] : Mme Sophie MARTIN (épouse DURAND) fonction : directrice des achats et des finances
[Délégation est donnée à] : M. Thomas DUPONT fonction : responsable de l'Unité budget et contrôle interne au sein de la Direction des achats et des finances

Article 15
[En cas d'absence ou d'empêchement de] : Non mentionné
[Délégation est donnée à] : Non mentionné

Analysez le texte suivant :

 
{extracted_text}"""
        
        analysis_prompt_template = ChatPromptTemplate.from_messages([
            ("human", prompt_text)
        ])
        
        prompt_value = analysis_prompt_template.invoke({})
        response = llm.invoke(prompt_value.to_messages())
        
        print("✅ Analyse terminée.")
        
        return {
            **state,
            "analysis_result": response.content
        }
        
    except Exception as e:
        return {
            **state,
            "error": f"Erreur lors de l'analyse: {str(e)}"
        }

# ===== LANGGRAPH WORKFLOW =====

def create_pdf_analysis_workflow():
    """Créer le workflow LangGraph"""
    workflow = StateGraph(PDFAnalysisState)
    
    # Ajouter les nœuds
    workflow.add_node("extraction", extraction_node)
    workflow.add_node("analysis", analysis_node)
    
    # Définir les transitions
    workflow.set_entry_point("extraction")
    workflow.add_edge("extraction", "analysis")
    workflow.add_edge("analysis", END)
    
    return workflow.compile()

# ===== FONCTION PRINCIPALE =====

def process_pdf(pdf_filename: str, start_page: int = None, end_page: int = None):
    """
    Traite un PDF avec le workflow LangGraph
    
    Args:
        pdf_filename (str): Nom du fichier PDF
        start_page (int): Première page à extraire (optionnel)
        end_page (int): Dernière page à extraire (optionnel)
    
    Returns:
        dict: Résultat final du workflow
    """
    # Créer le workflow
    workflow = create_pdf_analysis_workflow()
    
    # État initial
    initial_state = {
        "pdf_filename": pdf_filename,
        "start_page": start_page,
        "end_page": end_page,
        "extracted_text": None,
        "extracted_file_path": None,
        "analysis_result": None,
        "error": None
    }
    
    print("🚀 Démarrage du workflow LangGraph...")
    
    # Exécuter le workflow
    final_state = workflow.invoke(initial_state)
    
    return final_state

# ===== TEST ET EXÉCUTION =====

if __name__ == "__main__":
    # Test du workflow
    print("🧪 Test du workflow LangGraph PDF...")
    
    # Exemple d'utilisation
    result = process_pdf("2025.5.sante.pdf", 117, 131)
    
    if result.get("error"):
        print(f"❌ Erreur : {result['error']}")
    else:
        print(f"""
=== RÉSULTATS DU WORKFLOW ===

📁 PDF traité: {result['pdf_filename']}
📄 Pages analysées: {result.get('start_page', 'début')} à {result.get('end_page', 'fin')}
💾 Fichier extrait: {result.get('extracted_file_path', 'N/A')}

=== ANALYSE ===
{result.get('analysis_result', 'Aucun résultat')}

✅ Workflow terminé avec succès !
        """)

🧪 Test du workflow LangGraph PDF...
🚀 Démarrage du workflow LangGraph...
🔍 Exécution du nœud d'extraction...


NameError: name '__file__' is not defined